In [ ]:
import module_loader
import pandas as pd
from bookirds.curves import *
from bookirds.dual import Dual

## Forward zero coupon cross-gamma

In [ ]:
nodes = {
    datetime(2022, 1, 1): Dual(1, {"v0": 1}),
    datetime(2023, 1, 1): Dual(1, {"v1": 1}),
    datetime(2024, 1, 1): Dual(1, {"v2": 1}),
    datetime(2025, 1, 1): Dual(1, {"v3": 1}),
    datetime(2026, 1, 1): Dual(1, {"v4": 1}),
    datetime(2027, 1, 1): Dual(1, {"v5": 1}),
}
swaps = {
    Swap(datetime(2022,1,1), 12*1, 12, 12): 1.00,
    Swap(datetime(2022,1,1), 12*2, 12, 12): 1.10,
    Swap(datetime(2022,1,1), 12*3, 12, 12): 1.20,
    Swap(datetime(2022,1,1), 12*4, 12, 12): 1.25,
    Swap(datetime(2022,1,1), 12*5, 12, 12): 1.27,
}
s_cv = SolvedCurve(
    nodes=nodes, interpolation="log_linear",
    obj_rates=list(swaps.values()), 
    swaps=list(swaps.keys()),
    algorithm="levenberg_marquardt"
)
s_cv.iterate()   

In [ ]:
portfolio = Portfolio([
    Swap2(datetime(2022, 1, 1), 5*12, 12, 12, fixed_rate=25.0, notional=-8.3e8),
])

In [ ]:
dz, ds = portfolio.risk_fwd_zero_rates(s_cv)
df = pd.DataFrame({"dPdz": dz[:,0], "dPds": ds[:,0], "dPdr": (dz+ds)[:,0]})
df.style.format("{:,.0f}")

In [ ]:
ss, sz, zz = portfolio.cross_gamma(s_cv)

In [ ]:
G = np.block([[ss, sz, ss + sz],
              [sz.T, zz, sz.T + zz],
              [(ss + sz).T, (sz.T + zz).T, ss + sz + sz.T + zz]])
labels = ["s1", "s2", "s3", "s4", "s5", "z1", "z2", "z3", "z4", "z5", "r1", "r2", "r3", "r4", "r5"]
df = pd.DataFrame(G, index=labels, columns=labels)
df.style.format("{:,.1f}")

In [ ]:
dx = np.array([0.2, 0.3, 0.2, 0.1, -0.1,
               -1.1, -1.2, -1.1, -0.8, -0.5,
               -5.2, -4.1, -2, 2.4, 7.8])[:, np.newaxis]
np.matmul(np.matmul(dx[10:15, 0], G[10:15, 10:15]), dx[10:15, :])

In [ ]:
np.matmul(np.matmul(dx.T, G), dx) / 2

## Testing the Jacobian Calculations

In [ ]:
s_cv.grad_v_r

In [ ]:
s_cv.grad_r_v

In [ ]:
pd.DataFrame(np.matmul(s_cv.grad_v_r, s_cv.grad_r_v)).style

## Delta/Delta Cross-Gamma on a 5Y IRS expressed in Par space

In [ ]:
portfolio = Portfolio([
    Swap2(datetime(2022, 1, 1), 5*12, 12, 12, fixed_rate=1.270, notional=-100e6),
])

In [ ]:
names = ["1Y", "2Y", "3Y", "4Y", "5Y"]
ss, sz, zz = portfolio.cross_gamma(s_cv, swaps=True)
G = np.block([[ss, sz, ss + sz],
              [sz.T, zz, sz.T + zz],
              [(ss + sz).T, (sz.T + zz).T, ss + sz + sz.T + zz]])
labels = ["1Ys", "2Ys", "3Ys", "4Ys", "5Ys", 
          "1Yz", "2Yz", "3Yz", "4Yz", "5Yz", 
          "1Yr", "2Yr", "3Yr", "4Yr", "5Yr"]
df = pd.DataFrame(G, index=labels, columns=labels)
df.style.format("{:,.0f}").applymap(lambda v: "color: red;" if v < 0 else "")

In [ ]:
df2 = df.loc[["1Yr", "2Yr", "3Yr", "4Yr", "5Yr"], ["1Yr", "2Yr", "3Yr", "4Yr", "5Yr"]]
df2
df2["Total"] = df2.sum(axis=1)
df2.loc["Total", :] = df2.sum(axis=0)
df2.style.format("{:,.1f}").applymap(lambda v: "color: red;" if v < 0 else "")

Comparison with the simple formula

In [ ]:
df = pd.DataFrame(portfolio.risk(s_cv), index=names)
df.style.format("{:,.0f}").applymap(lambda v: "color: red;" if v < 0 else "")

$$\frac{D^2 P}{D \mathbf{r}^2} \approx - \frac{h+i}{10,000} S = \frac{6 * 48,260}{10,000} = 28.9 $$